# Feature Engineering

In [12]:
# Environment variables
from dotenv import load_dotenv
load_dotenv("mysql.env")

import os
import sys
import mysql.connector

import pandas as pd
import numpy as np
from ta import add_all_ta_features
from ta.utils import dropna

# List of Stocks and ETFs
Provided by Thomas Choi.

In [2]:
stock_list = pd.read_csv("stocks_and_etfs/stock_list.csv")
etf_list = pd.read_csv("stocks_and_etfs/etf_list.csv")

In [3]:
stock_symbol = stock_list.iloc[20,0]
stock_symbol

'VIAC'

# MySQL connection

In [4]:
HOST=os.environ.get("HOST")
PORT=os.environ.get("PORT")
USER=os.environ.get("USER")
PASSWORD=os.environ.get("PASSWORD")

try: 
    conn = mysql.connector.connect(
        host=HOST,
        port=PORT,
        user=USER,
        password=PASSWORD,
        database="GlobalMarketData"
    )
    query = f"SELECT Date, Close, Open, High, Low, Volume from histdailyprice3 WHERE Symbol='{stock_symbol}';"
    histdailyprice3 = pd.read_sql(query, conn)
    conn.close()
except Exception as e:
    conn.close()
    print(str(e))

# Load Data

In [19]:
df = histdailyprice3.copy()
df.set_index("Date", drop=True, inplace=True)

# Stock Dataset

In [20]:
# Clean NaN values
df = dropna(df)

# Add ta features filling NaN values
df = add_all_ta_features(
    df, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)

In [22]:
df.columns

Index(['Close', 'Open', 'High', 'Low', 'Volume', 'volume_adi', 'volume_obv',
       'volume_cmf', 'volume_fi', 'volume_mfi', 'volume_em', 'volume_sma_em',
       'volume_vpt', 'volume_nvi', 'volume_vwap', 'volatility_atr',
       'volatility_bbm', 'volatility_bbh', 'volatility_bbl', 'volatility_bbw',
       'volatility_bbp', 'volatility_bbhi', 'volatility_bbli',
       'volatility_kcc', 'volatility_kch', 'volatility_kcl', 'volatility_kcw',
       'volatility_kcp', 'volatility_kchi', 'volatility_kcli',
       'volatility_dcl', 'volatility_dch', 'volatility_dcm', 'volatility_dcw',
       'volatility_dcp', 'volatility_ui', 'trend_macd', 'trend_macd_signal',
       'trend_macd_diff', 'trend_sma_fast', 'trend_sma_slow', 'trend_ema_fast',
       'trend_ema_slow', 'trend_adx', 'trend_adx_pos', 'trend_adx_neg',
       'trend_vortex_ind_pos', 'trend_vortex_ind_neg', 'trend_vortex_ind_diff',
       'trend_trix', 'trend_mass_index', 'trend_cci', 'trend_dpo', 'trend_kst',
       'trend_kst_sig', '

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1124 entries, 2005-01-21 to 2021-09-03
Data columns (total 88 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Close                      1124 non-null   float64
 1   Open                       1124 non-null   float64
 2   High                       1124 non-null   float64
 3   Low                        1124 non-null   float64
 4   Volume                     1124 non-null   float64
 5   volume_adi                 1124 non-null   float64
 6   volume_obv                 1124 non-null   float64
 7   volume_cmf                 1124 non-null   float64
 8   volume_fi                  1124 non-null   float64
 9   volume_mfi                 1124 non-null   float64
 10  volume_em                  1124 non-null   float64
 11  volume_sma_em              1124 non-null   float64
 12  volume_vpt                 1124 non-null   float64
 13  volume_nvi                 1124 non-nu

In [24]:
# Descriptive statistics
df.describe()

Close         Open         High          Low        Volume  \
count  1124.000000  1124.000000  1124.000000  1124.000000  1.124000e+03   
mean     16.634327    16.998821    16.216437    16.598891  6.368148e+06   
std      16.669061    17.101092    16.250944    16.691785  1.451536e+07   
min       3.680000     3.720000     3.560000     3.660000  1.680000e+04   
25%       5.347500     5.490000     5.235000     5.330000  1.374295e+05   
50%       7.220000     7.245000     7.130000     7.200000  2.868820e+05   
75%      27.500000    28.115000    27.101250    27.685000  9.300975e+06   
max      99.150000   101.970000    96.210000   100.340000  2.165741e+08   

         volume_adi    volume_obv   volume_cmf     volume_fi   volume_mfi  \
count  1.124000e+03  1.124000e+03  1124.000000  1.124000e+03  1124.000000   
mean  -3.194494e+08 -6.433874e+07    -5.834508 -4.398003e+06    51.656557   
std    1.992082e+07  1.965061e+08     5.880422  4.577712e+07    20.076531   
min   -3.226996e+08 -6.960205e+08   -46.962864 -7.403318e+08     0.000000   
25%   -3.226864e+08 -5.640846e+07    -6.638736 -1.506666e+04    36.173431   
50%   -3.226864e+08  2.954816e+06    -4.513295  1.472039e+03    50.718567   
75%   -3.226864e+08  8.144266e+06    -2.445197  1.024396e+05    65.255961   
max   -5.451887e+07  4.869981e+08     0.624604  8.222377e+07   100.000000   

       ...  momentum_wr  momentum_ao  momentum_kama  momentum_roc  \
count  ...  1124.000000  1124.000000    1124.000000   1124.000000   
mean   ...   -32.427277     0.391482      16.577363      5.779590   
std    ...    69.400494     5.989693      16.480865     49.238463   
min    ...  -414.965986   -25.884737       3.945080    -55.763994   
25%    ...   -77.788400    -0.498126       5.402529     -7.786027   
50%    ...   -29.629630    -0.023059       7.218502      0.000000   
75%    ...     3.373756     0.850085      26.827731      7.569035   
max    ...  1666.666667    30.060453      90.612185    493.655172   

       momentum_ppo  momentum_ppo_signal  momentum_ppo_hist    others_dr  \
count   1124.000000          1124.000000        1124.000000  1124.000000   
mean      -3.516326            -3.483439          -0.032887     0.451871   
std       19.606289            17.004885           8.725381    14.694226   
min      -69.852645           -63.581800         -23.464178   -33.871685   
25%      -12.947378           -10.782003          -4.640794    -1.993854   
50%       -3.208875            -2.576071          -0.989282     0.000000   
75%        6.828985             5.809818           3.285488     1.781983   
max       85.155020            42.651885          71.869954   465.517241   

        others_dlr    others_cr  
count  1124.000000  1124.000000  
mean      0.118239    51.221153  
std       6.855726   151.536922  
min     -33.592486   -66.545455  
25%      -2.009279   -51.386364  
50%       0.000000   -34.363636  
75%       1.766292   150.000000  
max     173.257060   801.363636  

[8 rows x 88 columns]